In [ ]:
# from Libs.Profile.stocks import Stock, project_stock_price, plot_stock_value
# from Libs.Profile.portfolio import FinancialPortfolio
# import os
# from Libs.global_vars import STOCKS_HISTORY_PATH

# portfolio = FinancialPortfolio('verybigpack')

# # Load every file
# for file_stock in [f for f in os.listdir(STOCKS_HISTORY_PATH) if f.endswith('.csv')]:
#     portfolio.add_stock(Stock(stock_to_load=file_stock))
# # print(portfolio)
# portfolio.get_ammounts_with_risks()

# test = portfolio.get_stock('S&P500')
# test.update_stock(stock_prior_risk=0.02)
# # test.stock_history['date'] = ['2025-03-06 11:36:22', '2025-04-06 11:36:22', '2025-06-08 00:59:51', '2025-06-06 00:59:51', '2025-06-28 00:59:51', '2025-07-03 00:59:51', '2025-07-08 00:59:51', '2025-07-09 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51']
# import numpy as np
# for i in np.random.normal(50,2,5):
#     test.update_stock(stock_price=i)
# test.stock_history['date']=['2025-02-06 11:36:22', '2025-03-06 11:36:22', '2025-04-08 01:02:26', '2025-04-28 01:02:26', '2025-05-02 01:02:26', '2025-05-08 01:02:26', '2025-06-04 01:02:26', '2025-06-08 01:02:26']

# plot_stock_value(test)


## **Update Movements**

In [ ]:
# import polars as pl
# import numpy as np
# from Libs.Management.bank_api import get_history_movements, update_movements_file
# update_movements_file(new_orig_mov_file='data/raw_data/operazioni.xlsx', header_start=17)
# get_history_movements().head()

## **Update Balance**

In [ ]:
import polars as pl
import numpy as np
from Libs.Management.bank_api import get_current_balance, get_history_balance, reconstruct_balance, update_balance_from_movements
# reconstruct_balance(current_balance=to_give) # Only for creating the balance history
update_balance_from_movements()
get_history_balance().head()

# **Prediciton from Balance Example**

In [ ]:
from Libs.Models.data_visualization import plot_history_balance, plot_generic_polars
from Libs.Management.bank_api import get_history_balance
from Libs.Models.data_manipulation import get_regularized_dataset, get_moving_average, add_periodic_income

# Get Balance, regularize it and take the moving average
balance_df = get_history_balance()
df = get_regularized_dataset(balance_df, reg_period='1d')
df_ma = get_moving_average(df, window_size=15)

# Perform date to int conversion so that we can perform data imputation on the missing vals from moving average
from Libs.Models.data_manipulation import simple_linear_imputation, convert_date_to_numeric, convert_numeric_to_date
df_ma_imputed = convert_date_to_numeric(df_ma, ['date'])
df_ma_imputed = simple_linear_imputation(df_ma_imputed, 'date', 'moving_average')

# Perform a linear fit on the imputed dataset and go back to date format
from Libs.Models.Predictions.algorithms import linear_fit, fit_generic_polynomial_statsmodel
from Libs.Models.Predictions.algorithms import get_model_prediction, model_predict_statsmodel
# df_ma_imputed, linear_model = linear_fit(dataset=df_ma_imputed, cols_to_fit=None)
df_ma_imputed, linear_model = fit_generic_polynomial_statsmodel(dataset=df_ma_imputed, x_col='date', y_col='moving_average')
df_ma_imputed = convert_numeric_to_date(df_ma_imputed, ['date'])

# Define some new dates for forecasting and perform the prediction with the model created
X = convert_date_to_numeric(['2025-06-01', '2025-07-01', '2025-08-01', '2030-08-01'])
# pred_df = get_model_prediction(X, linear_model)
pred_df, prediction = model_predict_statsmodel(X=X, model=linear_model)

pred_df = convert_numeric_to_date(pred_df, date_cols=['date'])

# Show the results
plot_history_balance(balance_df=df)
plot_generic_polars(dataset=df_ma, x_col='date', y_col='moving_average', params={'figsize':(10,5), 'label':'moving average', 'marker':'', 'linewidth':3, 'color':'darkblue'})
df_ma_imputed_new = add_periodic_income(1500.0, day=15, periodicity='month', start_date='2024-01-01', end_date='2024-06-01')
plot_generic_polars(dataset=df_ma_imputed, x_col='date', y_col='moving_average', params={'figsize':(10,5), 'label':'imputed data', 'marker':'', 'linewidth':1, 'color':'red'})
plot_generic_polars(dataset=pred_df, x_col='date', y_col='model_prediction', params={'figsize':(10,5), 'label':'prediction', 'color':'darkred', 'confidence_belt':['mean_ci_lower', 'mean_ci_upper']})

from Libs.Models.data_visualization import get_uniformed_list, plot_generic_timeseries
uniformed_vals = get_uniformed_list([df_ma_imputed['moving_average'].to_list(), pred_df['model_prediction'].to_list()])
fig = plot_generic_timeseries([None]*len(uniformed_vals), uniformed_vals, params={'xlabel':'dates', 'ylabel':'EUR', 'title':'Plot Simulation'})


In [ ]:

from Libs.Models.data_manipulation import add_periodic_income
# Using string dates for custom date range
custom_income1 = add_periodic_income(1500.0, day=15, periodicity='month', start_date='2024-01-01', end_date='2024-06-01')
custom_income2 = add_periodic_income(1111.0, day=15, periodicity='month', start_date='2024-01-03', end_date='2024-07-06')



In [ ]:
df_ma['date'].to_list()[0].strftime("%Y-%m-%d")


In [ ]:
pred_df

In [ ]:
linear_model['model'].

In [ ]:
from Libs.Models.Predictions.algorithms import goal_reach_prediction
goal_reach_prediction(linear_model, '2025-07-01', 15_000)

In [ ]:
all([None, 1, None])

In [ ]:
from Libs.Models.data_visualization import get_uniformed_list, plot_generic_timeseries
import numpy as np

range1 = ['2025-09-01', '2025-10-02', '2025-11-3','2025-11-04','2025-11-05']
vals1 = np.random.rand(len(range1))*10+np.linspace(0,10, len(range1))

range2 = ['2025-10-03', '2025-10-11', '2025-11-02', '2025-11-04']
vals2 = np.random.rand(len(range2))+np.linspace(0,10, len(range2))

plot_generic_timeseries(range1, vals1, params={'label':'vals1', 'color':'black'})
plot_generic_timeseries(range2, vals2, params={'label':'vals2', 'color':'red'})

uniformed_x = get_uniformed_list([range1,range2])
fig = plot_generic_timeseries(uniformed_x, [None]*len(uniformed_x), params={'xlabel':'dates', 'ylabel':'EUR', 'title':'Plot Simulation'})
fig.set_size_inches((10, 6))
fig.show()